## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem #импорт библиотеки pymystem3
m = Mystem()

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Датасет состоит из 21525 строк и 12 столбцов. В столбцах days_employed и total_income одинаковое количество пропусков.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#Заменяем все отрицательные числа на положительные
data.loc[data['days_employed'] < 0, 'days_employed'] = - data['days_employed']

#Заменим пропущенные значения на средние арифметические

data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('mean'))

#Функция transform возвращает датафрейм такого же размера, как и исходный

data.loc[data['children'] < 0, 'children'] = - data['children']
data.loc[data['children'] > 10, 'children'] =  2

### Вывод

В столбцах days_employed и total_income пропуски находятся в одинаковых строках. Пропусков не очень много, скорее всего произошла какая-то техническая ошибка и эти данные были утеряны. На наше исследование стаж и доход не сильно влияют и эти данные не влияют на информацию в других столбцах, поэтому эти данные можно было бы заменить на 0. Но для дальнейшей работы с датасетом лучше заменить пропуски, например, на средний стаж и доход для каждой категории граждан.
На вычисление среднего стажа влияют отрицательные значения. Скорее всего это тоже ошибка и поэтому я их заменил на положительные, равные по модулю.

Также оказалось, что в столбце chidren есть 76 человек, у которых 20 детей и 47 человек, у которых -1 ребенок, что несомненно является ошибкой. Поэтому 20 заменяем на 2, а -1 на 1

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Вывод

Заменяем значения float на int в столбцах со стажем и доходом. Для дальнейшей работы визуально удобнее работать с целыми числами. Тем более, стаж не может быть дробным. Используем метод astype, так как именно с помощью него можно перевести данные в целочисленный вид.

### Обработка дубликатов

In [5]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data = data.drop_duplicates().reset_index(drop = True)

### Вывод

В столбце education присутствовали значения в разном регистре. После приведения к одному регистру, дубликатов оказалось 71, что очень мало для такого большого датасета. Поэтому их можно удалить методом drop_duplicates в связке с reset_index. Я думаю, что дубликаты появились случайно. Может быть кто-то случайно внес одни данные несколько раз или люди брали кредиты несколько раз.

### Лемматизация

In [6]:
#from pymystem3 import Mystem # код ревьюера
#m = Mystem() # код ревьюера 

purpose_list = str(data['purpose'].unique())
purpose_list = m.lemmatize(purpose_list)

from collections import Counter

print(Counter(purpose_list)) 


Counter({' ': 59, "' '": 23, "'\n": 14, " '": 14, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, "['": 1, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, "']\n": 1})


In [7]:
purpose_list = str(data['purpose'].unique()) #найдем уникальные значения столбца purpose, приведем их к строчному виду и сохраним в переменной purpose_list



data['purpose_lemmas'] = data['purpose'].apply(lambda x: [(' '.join(m.lemmatize(x)))])



def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  
data['purpose_lemmas'] = data['purpose_lemmas'].apply(listToString)

def get_cat(row):
    if 'свадьба' in row:
        return 'свадьба'
    elif 'жилье' in row:
        return 'жилье'
    elif 'недвижимость' in row:
        return 'недвижимость'
    elif 'образование' in row:
        return 'образование'
    elif 'автомобиль' in row:
        return 'автомобиль'
    else:
        return 0
data['purpose_lemmas'] = data['purpose_lemmas'].apply(lambda x: get_cat(x))
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


### Вывод

После подсчета слов в лемматизированном списке уникальных значений для столбца purpose, можно выделить 5 лемм: свадьба, недвижимость, автомобиль, образование, жилье. Есть смысл добавить в датасет столбец, который описывает каждую цель одним словом, чтобы проще было группировать и категоризировать.

### Категоризация данных

In [8]:
data_log = data[['children', 'days_employed', 'dob_years','education_id','family_status_id', 'gender',
                'income_type', 'debt', 'total_income', 'purpose_lemmas']]
data_dict_purpose = data[['purpose_lemmas', 'purpose']]
data_dict_family_status = data[['family_status_id', 'family_status']]
data_dict_education = data[['education_id', 'education']]

data_dict_purpose = data_dict_purpose.drop_duplicates().reset_index(drop=True)
data_dict_family_status = data_dict_family_status.drop_duplicates().reset_index(drop=True)
data_dict_education = data_dict_education.drop_duplicates().reset_index(drop=True)

### Вывод

Если посмотреть на названия наших столбцов, можно понять, что можно выделить три различных словаря: цели, семейное положение и уровень образование. Оставим тогда просто численные значения этих столбцов в таблице, а "расшифровки" отправим в словарь. Потом с помощью метода drop_duplicates избавимся от дубликатов.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
def childif(row):
    if row == 0:
        return 'нет'
    else:
        return 'есть'

def have_child_counter(row):
    if 'есть' in row:
        return 1
    else:
        return 0

data['children_detector'] = data['children'].apply(childif)
sum_have_children = data[data['children_detector'] == 'есть']['children_detector'].apply(have_child_counter).sum()
debt_have_children = data[data['children_detector'] == 'есть']['debt'].sum()

print('Абсолютные значения для количества долгов:')
print(data_log.groupby('children')['debt'].sum())

#Посчитаем отношение должников к общему числу для каждой группы по детям
print('Доля должников для различного количества детей:')
print(data_log.groupby('children')['debt'].sum() /  data_log['children'].value_counts())

#Интересно еще посмотреть на долю должников среди всех людей с детьми
print('Доля должников среди всех людей с детьми:', debt_have_children / sum_have_children)


Абсолютные значения для количества долгов:
children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64
Доля должников для различного количества детей:
children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
dtype: float64
Доля должников среди всех людей с детьми: 0.09208203178052424


In [16]:
data.groupby('children').agg({'debt': ['count', 'sum', 'mean']}).style.format({('debt', 'mean'): '{:.2%}'})

### Вывод

Можем сделать вывод, что у бездетных людей больше долгов по кредитам. Однако их значительно больше, чем людей с детьми. Поэтому, если мы посмотрим в процентном соотношении, люди с детьми отдают кредиты реже.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
print('Абсолютные значения для количества долгов:')
print(data.groupby('family_status')['debt'].sum())

print('Доля должников среди всех людей с детьми:')
print(data.groupby('family_status')['debt'].sum() / data['family_status'].value_counts())


Абсолютные значения для количества долгов:
family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
не женат / не замужем    274
Name: debt, dtype: int64
Доля должников среди всех людей с детьми:
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
не женат / не замужем    0.097509
dtype: float64


### Вывод

У людей, состоящих в браке, больше долгов по кредитам. Но процент должников со статусами не женат и гражданский брак чуть выше остальных.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
def income_sort(row):
    if row <= 50000:
        return 'низкий'
    if row > 50000 and row <= 150000:
        return 'средний'
    if row > 150000 and row <= 250000:
        return 'выше среднего'
    if row > 250000:
        return 'высокий'
data['type_income'] = data['total_income'].apply(income_sort)

print('Абсолютные значения для количества долгов:')
print(data.groupby('type_income')['debt'].sum())


print('Доля от всех должников:')
print(data.groupby('type_income')['debt'].sum() / data['type_income'].value_counts())

Абсолютные значения для количества долгов:
type_income
высокий          194
выше среднего    667
низкий            23
средний          857
Name: debt, dtype: int64
Доля от всех должников:
высокий          0.068966
выше среднего    0.082468
низкий           0.061828
средний          0.084176
dtype: float64


### Вывод

Больше всего долгов по кредитам у людей с доходом от 50000 до 250000. Они же главные должники, если смотреть долю от всех должников.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
print('Количество долгов для разных целей кредита:')
print(data.groupby('purpose_lemmas')['debt'].sum())

print('Доля каждой категории:')
print(data.groupby('purpose_lemmas')['debt'].sum() / data['purpose_lemmas'].value_counts())

Количество долгов для разных целей кредита:
purpose_lemmas
автомобиль      403
жилье           308
недвижимость    474
образование     370
свадьба         186
Name: debt, dtype: int64
Доля каждой категории:
автомобиль      0.093590
жилье           0.069058
недвижимость    0.074634
образование     0.092200
свадьба         0.080034
dtype: float64


### Вывод

Доля должников, которые взяли кредит на автомобиль и образование больше, чем доля людей с другими целями кредита.

### Шаг 4. Общий вывод

Для ответа на главный вопрос исследования, была проведена подготовка исходного датасета к исследованию. 

Были заполнены пропуски в столбцах "стаж" и "доход" на средние значения по каждой категории. Были удалены такие артефакты как отрицательный стаж, отрицательное количество детей и неправдоподобное количество детей у некоторых людей. Скорее всего, эти ошибки связаны либо со сбоем в системе, либо с человеческим фактором.

После приведения всех слов в таблице к нижнему регистру, были удалены дубликаты.

В процессе лемматизации были выделены 5 главных категорий целей кредита для более удобной работы с датасетом.

Была проведена категоризация для более удобной работы с таблицей.

В результате, можно сделать следующие выводы:
1. Люди с детьми реже отдают кредиты, чем бездетные;
2. Холостые люди и люди, проживающие в гражданском браке - не самые надежные заемщики;
3. Удивительно, но у большинства должников не самый низкий доход. Обычно, должники - представители "среднего класса";
4. Выдавать кредиты на покупку автомобиля или на образование - довольно рисковая идея.